In [ ]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------
# 1단계 기술 실행 계획: 데이터 수집 및 전처리 파이프라인 - 셀 1 (AbNumber 사용 버전)
# Target: AI 기반 항체 생성/최적화 벤처 (한국)
# Based on the user's detailed plan document and subsequent feedback.
# Using AbNumber, handling specific data paths and formats.
# -----------------------------------------------------------

# 1.1. 라이브러리 설치
print("Installing condacolab...")
# Colab에는 Conda 환경이 기본 설치되어 있지 않으므로 필요 시 설치
!pip install -q condacolab
import condacolab
condacolab.install() # 런타임 다시 시작 필요할 수 있음

# AbNumber 및 기타 필수 라이브러리 설치
print("Installing AbNumber and other Python libraries via pip...")
# 최신 pip 및 setuptools 사용 권장
!conda install bioconda::abnumber
!pip install --upgrade pip setuptools wheel
!pip install pandas requests tqdm pyarrow aiohttp biopython ipython # IPython은 display용
!pip install abnumber # AbNumber 설치

print("Library installation finished.")

# 1.2. 라이브러리 임포트
import requests
import pandas as pd
import os
import time
import logging
import warnings
import tarfile
import zipfile
import gzip
import shutil
import asyncio
import aiohttp
from pathlib import Path
from Bio.PDB import PDBParser, Polypeptide, PDBIO, Select
from Bio.SeqIO import FastaIO
from Bio import SeqIO, BiopythonWarning
from Bio.SeqUtils import seq1
from Bio.PDB.PDBExceptions import PDBConstructionWarning
from tqdm.auto import tqdm
import sys
from collections import defaultdict # defaultdict 추가
from IPython.display import display # Colab에서 DataFrame 보기 좋게 출력
import abnumber # AbNumber 라이브러리 임포트
from abnumber.exceptions import ChainParseError # AbNumber의 특정 예외 처리

# 1.3. 구글 드라이브 마운트
print("Mounting Google Drive...")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Google Drive 마운트 중 오류 발생: {e}")

# 1.4. 기본 설정 및 상수 정의
# ================== 사용자 설정 영역 START ==================
print("Configuring constants...")
# === 결과 저장 경로 ===
DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber" # 사용자 경로 반영

# === 테스트 및 리소스 관리 ===
#SUBSET_SIZE = 50 # 테스트 시: 작은 값 설정 (예: 50)
SUBSET_SIZE = None # 전체 데이터 처리 시: None 설정

# === 품질 관리 기준 ===
MIN_RESOLUTION = 3.0
MIN_VH_LEN = 80; MAX_VH_LEN = 150
MIN_VL_LEN = 80; MAX_VL_LEN = 150
STANDARD_AMINO_ACIDS = "ACDEFGHIKLMNPQRSTVWY"

# === 데이터 소스 URL ===
SABDAB_SUMMARY_URL = "https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabdab/summary/all/"
PDB_DOWNLOAD_URL = "https://files.rcsb.org/download/{}.pdb"
ABSD_BULK_URL = "https://absd.pasteur.cloud/api/downloads/2025-03-24.tar.gz" # 유효성 확인 필요

# === 네트워크 설정 ===
REQUEST_TIMEOUT = 300
DOWNLOAD_SLEEP_INTERVAL = 0.3 # 비동기에서는 사용 안 함
MAX_DOWNLOAD_RETRIES = 3
DOWNLOAD_CONCURRENCY = 20 # PDB 다운로드 동시성 증가

print("Constants configured.")
# ================== 사용자 설정 영역 END ==================

# --- 파생 경로 설정 ---
print("Setting up derived paths...")
RAW_DATA_DIR = os.path.join(DRIVE_OUTPUT_DIR, "0_raw_data")
PREPROCESSED_DIR = os.path.join(DRIVE_OUTPUT_DIR, "1_preprocessed")
FINAL_DATA_DIR = os.path.join(PREPROCESSED_DIR, "final_dataset")
LOG_FILE = os.path.join(DRIVE_OUTPUT_DIR, "data_pipeline_abnumber.log") # 로그 파일명 변경
SABDAB_RAW_DIR = os.path.join(RAW_DATA_DIR, "sabdab"); PDB_RAW_DIR = os.path.join(RAW_DATA_DIR, "pdb")
ABSD_RAW_DIR = os.path.join(RAW_DATA_DIR, "absd"); AACDB_RAW_DIR = os.path.join(RAW_DATA_DIR, "aacdb")
OAS_RAW_DIR = os.path.join(RAW_DATA_DIR, "oas"); FINAL_SEQS_DIR = os.path.join(FINAL_DATA_DIR, "sequences")
FINAL_PDB_DIR = os.path.join(FINAL_DATA_DIR, "structures")
FINAL_METADATA_FILE = os.path.join(FINAL_DATA_DIR, "antibody_metadata_abnumber.parquet") # 결과 파일명 변경

# --- AACDB 파일 경로 (사용자 확인 경로 기반) ---
AACDB_SUMMARY_FILE = os.path.join(AACDB_RAW_DIR, "aacdb_summary.txt")
AACDB_ANTIBODY_FASTA = os.path.join(AACDB_RAW_DIR, "aacdb_antibody_seqs.fasta")
AACDB_INTERACTION_SASA_DIR = os.path.join(AACDB_RAW_DIR, "extracted_interaction_sasa")
AACDB_INTERACTION_DIST_DIR = os.path.join(AACDB_RAW_DIR, "extracted_interaction_dist")
AACDB_PDB_DIR = os.path.join(AACDB_RAW_DIR, "extracted_complex_pdbs") # 사용자 확인 경로

print("Derived paths set.")

# 1.5. 로깅 설정
print("Setting up logging...")
for handler in logging.root.handlers[:]: logging.root.removeHandler(handler)
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)-8s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE, mode='w', encoding='utf-8'),
        logging.StreamHandler(sys.stdout)
    ]
)
logging.info("="*50); logging.info(" 항체 데이터 수집 및 전처리 파이프라인 시작 (AbNumber + Multi-Source)"); logging.info("="*50)
logging.info(f"결과 저장 경로: {DRIVE_OUTPUT_DIR}")
if SUBSET_SIZE: logging.warning(f"!!! 테스트 모드 활성화: 각 소스별 최대 {SUBSET_SIZE}개 항목만 처리합니다 !!!")
else: logging.warning("!!! 전체 데이터 처리 모드: 상당한 시간과 리소스가 필요합니다 !!!")
print("Logging setup complete.")

# 1.6. 경고 메시지 제어
print("Configuring warnings...")
warnings.simplefilter('ignore', PDBConstructionWarning)
warnings.simplefilter('ignore', BiopythonWarning)
print("Warnings configured.")

# 1.7. 디렉토리 생성 함수 정의
print("Defining setup_directories function...")
def setup_directories():
    """필요한 모든 출력 디렉토리를 생성합니다."""
    logging.info("출력 디렉토리 설정 중...")
    dirs_to_create = [ DRIVE_OUTPUT_DIR, RAW_DATA_DIR, PREPROCESSED_DIR, FINAL_DATA_DIR,
                       SABDAB_RAW_DIR, PDB_RAW_DIR, ABSD_RAW_DIR, AACDB_RAW_DIR, OAS_RAW_DIR,
                       FINAL_SEQS_DIR, FINAL_PDB_DIR,
                       # AACDB/ABSD 추출 경로도 확인
                       os.path.join(AACDB_RAW_DIR, "extracted_complex_pdbs"),
                       os.path.join(AACDB_RAW_DIR, "extracted_interaction_sasa"),
                       os.path.join(AACDB_RAW_DIR, "extracted_interaction_dist"),
                       os.path.join(ABSD_RAW_DIR, "extracted"),
                       os.path.join(ABSD_RAW_DIR, "extracted", "data") # ABSD data 하위 폴더
                      ]
    for dir_path in dirs_to_create:
        try:
             os.makedirs(dir_path, exist_ok=True)
        except OSError as e:
             logging.error(f"디렉토리 생성 실패: {dir_path} - {e}")
    logging.info("출력 디렉토리 설정 완료.")
print("setup_directories function defined.")

print("\n--- 셀 1 실행 완료 ---")

Installing condacolab...
✨🍰✨ Everything looks OK!
Installing AbNumber and other Python libraries via pip...
Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Library installation finished.
Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Configuring constants...
Constants configured.
Setting up derived paths...
Derived paths set.
Setting up logging...
2025-04-27 12:36:52,969 - INFO     - ==================================================
2025-04-27 12:36:52,970 - INFO     -  항체 데이터 수집 및 전처리 파이프라인 시작 (AbNumber + Multi-Source)
2025-04-27 12:36:52,973 - INFO     - ===========

In [ ]:
# ABSD 파일 확인
!ls -l /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/absd/absd_all_species.tar.gz

# OAS 파일 확인 (상위 몇 개만)
!ls -lh /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/oas/*.csv.gz | head

# AACDB 파일 확인
!ls -l /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/aacdb/aacdb_summary.txt
!ls -l /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/aacdb/aacdb_antibody_seqs.fasta

# AACDB 압축 파일 확인 (예시) - 실제 파일 이름 확인 필요
!ls -l /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/aacdb/Complex_PDBs.zip
!ls -l /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/aacdb/interaction_sasa.zip
!ls -l /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/aacdb/interaction_dist.zip

-rw------- 1 root root 138895047 Apr 24 08:16 /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/absd/absd_all_species.tar.gz
-rw------- 1 root root  8.6M Nov 16  2022 /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/oas/1279049_1_Paired_All.csv.gz
-rw------- 1 root root   18M Nov 16  2022 /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/oas/1279050_1_Paired_All.csv.gz
-rw------- 1 root root   14M Nov 16  2022 /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/oas/1279051_1_Paired_All.csv.gz
-rw------- 1 root root  1.4M Nov 16  2022 /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/oas/1279052_1_Paired_All.csv.gz
-rw------- 1 root root   11M Nov 16  2022 /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/oas/1279053_1_Paired_All.csv.gz
-rw------- 1 root root   11M Nov 16  2022 /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/oas/1279054_1_Paired_All.csv.gz
-rw------- 1

In [ ]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------
# 2. 헬퍼 함수 정의 - 전체 코드 (AbNumber + 경로 수정 반영)
# -----------------------------------------------------------
import os
import requests
import asyncio
import aiohttp
from tqdm.auto import tqdm
import time
import logging
import shutil
from pathlib import Path
from Bio.PDB import Select, Polypeptide, PDBParser, PDBIO
from Bio.SeqUtils import seq1
import abnumber
from abnumber.exceptions import ChainParseError
from typing import Optional, Dict, Tuple, List
import traceback
import json
import pandas as pd

# --- 파일 다운로드 함수 ---
# ... (download_file_async, download_files_in_batch, download_file_sync 함수 코드는 변경 없음) ...
async def download_file_async(session, url, dest_path, desc="파일 다운로드 중"):
    """aiohttp를 사용하여 비동기적으로 파일을 다운로드합니다."""
    retries = 0
    while retries < MAX_DOWNLOAD_RETRIES:
        try:
            async with session.get(url, timeout=REQUEST_TIMEOUT) as response:
                response.raise_for_status()
                total_size = int(response.headers.get('content-length', 0))
                block_size = 1024 * 8
                progress_bar = tqdm(total=total_size, unit='B', unit_scale=True, desc=f"{desc} ({os.path.basename(dest_path)})", leave=False)
                with open(dest_path, 'wb') as f:
                    while True:
                        chunk = await response.content.read(block_size)
                        if not chunk: break
                        f.write(chunk)
                        progress_bar.update(len(chunk))
                progress_bar.close()
                actual_size = os.path.getsize(dest_path)
                log_level = logging.DEBUG if "rcsb.org" in url else logging.WARNING
                if total_size != 0 and actual_size != total_size:
                     logging.log(log_level, f"다운로드 크기 불일치: {url} -> {dest_path} (예상: {total_size}, 실제: {actual_size})")
                return dest_path
        except (aiohttp.ClientError, asyncio.TimeoutError) as e:
            retries += 1; logging.warning(f"{url} 다운로드 오류 (시도 {retries}/{MAX_DOWNLOAD_RETRIES}): {e}. 잠시 후 재시도..."); await asyncio.sleep(2 ** retries)
        except Exception as e:
            logging.error(f"{url} 다운로드 중 예상치 못한 오류: {e}")
            if os.path.exists(dest_path):
                try: os.remove(dest_path)
                except OSError as oe: logging.warning(f"부분 파일 삭제 실패 ({dest_path}): {oe}")
            return None
    logging.error(f"{url} 다운로드 최종 실패 ({MAX_DOWNLOAD_RETRIES}회 시도).")
    return None

async def download_files_in_batch(urls_and_paths, desc="파일 배치 다운로드", concurrency=10):
    """주어진 URL과 저장 경로 목록을 비동기 배치로 다운로드합니다."""
    connector = aiohttp.TCPConnector(limit=concurrency, ssl=False)
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = [download_file_async(session, url, path, desc) for url, path in urls_and_paths]
        results = await asyncio.gather(*tasks)
        return results

def download_file_sync(url, dest_path, desc="파일 다운로드 중"):
    """requests를 사용하여 동기적으로 파일을 다운로드합니다."""
    retries = 0
    while retries < MAX_DOWNLOAD_RETRIES:
        try:
            response = requests.get(url, timeout=REQUEST_TIMEOUT, stream=True)
            response.raise_for_status()
            total_size = int(response.headers.get('content-length', 0))
            block_size = 1024 * 8
            progress_bar = tqdm(total=total_size, unit='B', unit_scale=True, desc=f"{desc} ({os.path.basename(dest_path)})")
            with open(dest_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=block_size):
                    if chunk: f.write(chunk); progress_bar.update(len(chunk))
            progress_bar.close()
            actual_size = os.path.getsize(dest_path)
            log_level = logging.DEBUG if "rcsb.org" in url else logging.WARNING
            if total_size != 0 and actual_size != total_size:
                 logging.log(log_level, f"다운로드 크기 불일치: {url} -> {dest_path} (예상: {total_size}, 실제: {actual_size})")
            return dest_path
        except (requests.exceptions.RequestException, requests.exceptions.Timeout) as e:
            retries += 1; logging.warning(f"{url} 다운로드 오류 (시도 {retries}/{MAX_DOWNLOAD_RETRIES}): {e}. 잠시 후 재시도..."); time.sleep(2 ** retries)
        except Exception as e:
            logging.error(f"{url} 다운로드 중 예상치 못한 오류: {e}")
            if os.path.exists(dest_path):
                try: os.remove(dest_path)
                except OSError as oe: logging.warning(f"부분 파일 삭제 실패 ({dest_path}): {oe}")
            return None
    logging.error(f"{url} 다운로드 최종 실패 ({MAX_DOWNLOAD_RETRIES}회 시도).")
    return None


# --- AbNumber 실행 함수 (잔기 추출 로직 수정됨 - v4) ---
def run_abnumber(sequence, scheme='imgt'):
    """
    주어진 아미노산 서열에 대해 AbNumber 라이브러리를 사용하여 번호를 매기고 CDR을 추출합니다.
    번호 매겨진 잔기 추출 방식을 chain.positions 직접 접근으로 개선 시도.
    반환: (numbered_residues, cdr_sequences, status_msg)
    """
    numbered_residues = {}
    cdr_sequences = {}
    status_msg = "AbNumber_ProcessingFailed"

    if not sequence or not isinstance(sequence, str):
        return None, None, "AbNumber_Error_InvalidInputSequence"

    try:
        chain = abnumber.Chain(sequence, scheme=scheme)
        logging.debug(f"Chain type: {chain.chain_type}, Seq: {sequence[:30]}...")

        # --- 수정: 번호 매겨진 잔기 추출 시도 (chain.positions 우선) ---
        if hasattr(chain, 'positions') and isinstance(chain.positions, dict):
             numbered_residues = chain.positions
             # 갭(-) 제거 (필요시)
             numbered_residues = {pos: res for pos, res in numbered_residues.items() if isinstance(res, str) and res != '-'}
             logging.debug(f"Extracted {len(numbered_residues)} residues via chain.positions")
        else:
             # .positions 속성이 없거나 dict가 아니면 이전의 반복 방식 시도 (Fallback)
             logging.warning("chain.positions not found or not a dict, attempting iteration...")
             for item in chain:
                 pos_tuple, amino_acid = None, None
                 if hasattr(item, 'index') and hasattr(item, 'residue'): pos_tuple, amino_acid = item.index, item.residue
                 elif isinstance(item, tuple) and len(item) >= 2 and hasattr(item[0], 'index'): pos_tuple, amino_acid = item[0].index, item[1]
                 else: continue
                 if isinstance(pos_tuple, tuple) and len(pos_tuple) == 2 and isinstance(pos_tuple[0], int) and isinstance(pos_tuple[1], str) and isinstance(amino_acid, str) and amino_acid != '-':
                     numbered_residues[pos_tuple] = amino_acid
             logging.debug(f"Extracted {len(numbered_residues)} residues via iteration")
        # --- 수정 끝 ---


        # CDR 서열 추출 (이전과 동일하게 getattr 사용)
        cdr_sequences = {
            'CDR1': getattr(chain, 'cdr1_seq', None),
            'CDR2': getattr(chain, 'cdr2_seq', None),
            'CDR3': getattr(chain, 'cdr3_seq', None)
        }
        cdr_sequences = {k: v for k, v in cdr_sequences.items() if v is not None}

        # 상태 메시지 결정 (numbered_residues가 채워졌는지 확인)
        if numbered_residues and cdr_sequences: status_msg = "Success"
        elif numbered_residues: status_msg = "Success_NoCDR" # 번호는 매겼지만 CDR 없음
        elif chain.chain_type is not None: status_msg = "AbNumber_Success_EmptyResult" # 타입은 알지만 잔기 추출 실패
        else: status_msg = "AbNumber_ParseError_FailedCompletely" # 타입도 모르고 잔기도 없음

        logging.debug(f"AbNumber result: Status='{status_msg}', SeqLen={len(sequence)}, NumRes={len(numbered_residues)}, CDRs={bool(cdr_sequences)}")
        return numbered_residues, cdr_sequences, status_msg

    except ChainParseError as e: logging.warning(f"AbNumber ChainParseError (len {len(sequence)}): {e}"); return None, None, f"AbNumber_ParseError: {str(e)[:100]}"
    except ValueError as e: logging.warning(f"AbNumber ValueError (scheme='{scheme}'?): {e}"); return None, None, f"AbNumber_ValueError: {str(e)[:100]}"
    except AttributeError as e: logging.error(f"AbNumber AttributeError (len {len(sequence)}): {e}"); return None, None, f"AbNumber_AttributeError: {str(e)[:100]}"
    except Exception as e: logging.error(f"AbNumber unexpected error (len {len(sequence)}): {e}"); return None, None, f"AbNumber_UnknownError: {str(e)[:100]}"


# --- PDB 파일에서 서열 추출 함수 (이전과 동일) ---
def extract_vh_vl_sequences_from_pdb(pdb_path, h_chain, l_chain):
    """주어진 PDB 파일과 chain ID에서 VH, VL 서열을 추출합니다."""
    parser = PDBParser(QUIET=True)
    vh_seq, vl_seq = "", ""
    try:
        structure = parser.get_structure(f"pdb_{os.path.basename(pdb_path)}", pdb_path)
        if not structure: return None, None
        model = structure[0]
        if h_chain in model:
            for residue in model[h_chain]:
                if residue.id[0] == ' ' and Polypeptide.is_aa(residue.get_resname(), standard=True):
                    try: vh_seq += seq1(residue.get_resname())
                    except KeyError: pass
        else: logging.debug(f"{os.path.basename(pdb_path)}: H chain '{h_chain}' 찾을 수 없음."); return None, None
        if l_chain in model:
             for residue in model[l_chain]:
                if residue.id[0] == ' ' and Polypeptide.is_aa(residue.get_resname(), standard=True):
                    try: vl_seq += seq1(residue.get_resname())
                    except KeyError: pass
        else: logging.debug(f"{os.path.basename(pdb_path)}: L chain '{l_chain}' 찾을 수 없음."); return None, None
        if not vh_seq or not vl_seq: return None, None
        return vh_seq.upper(), vl_seq.upper()
    except FileNotFoundError: logging.warning(f"PDB 파일 접근 불가 (서열 추출 시): {pdb_path}"); return None, None
    except ValueError as ve: logging.warning(f"Error parsing PDB {os.path.basename(pdb_path)}: {ve}"); return None, None
    except Exception as e: logging.warning(f"Error extracting sequences from {os.path.basename(pdb_path)} ({h_chain}/{l_chain}): {e}"); return None, None


# --- Interaction Parsing Helper Functions (AACDB용 - 이전과 동일) ---
def parse_single_sasa_file(sasa_file_path):
    """단일 PDB 항목에 대한 SASA 상호작용 파일(.txt)을 파싱합니다."""
    interaction = {'antibody': [], 'antigen': []}
    antibody_chains_guess = {'H', 'L'}
    try:
        with open(sasa_file_path, 'r', encoding='utf-8', errors='ignore') as f: lines = f.readlines()
        header_skipped = False
        for line in lines:
            line = line.strip(); parts = line.split()
            if not line or line.startswith('#'): continue
            if not header_skipped and 'chain' in line.lower() and 'residues' in line.lower(): header_skipped = True; continue
            if not header_skipped: continue
            if len(parts) == 4:
                chain, res_name, pos, _ = parts; res_str = f"{chain}:{res_name.upper()}{pos}"
                if chain in antibody_chains_guess: interaction['antibody'].append(res_str)
                else: interaction['antigen'].append(res_str)
        return interaction if interaction['antibody'] or interaction['antigen'] else None
    except Exception as e: logging.error(f"SASA 파일({sasa_file_path}) 파싱 오류: {e}"); return None

def parse_single_distance_file(dist_file_path):
    """단일 PDB 항목에 대한 거리 기반 상호작용 파일(.txt or .dict)을 파싱합니다."""
    interaction = {'antibody': set(), 'antigen': set()}
    try:
        with open(dist_file_path, 'r', encoding='utf-8', errors='ignore') as f: lines = f.readlines()
        header_skipped = False
        for line in lines:
            line = line.strip(); parts = line.split()
            if not line or line.startswith('#'): continue
            if not header_skipped and 'antibody' in line.lower() and 'antigen' in line.lower(): header_skipped = True; continue
            if not header_skipped: continue
            if len(parts) >= 2:
                ab_res_str, ag_res_str = parts[0], parts[1]
                if ':' in ab_res_str and ':' in ag_res_str:
                    interaction['antibody'].add(ab_res_str); interaction['antigen'].add(ag_res_str)
        return interaction if interaction['antibody'] or interaction['antigen'] else None
    except Exception as e: logging.error(f"Distance 파일({dist_file_path}) 파싱 오류: {e}"); return None

logging.info("헬퍼 함수 정의 완료 (run_abnumber v4 적용).")

2025-04-27 12:36:55,162 - INFO     - 헬퍼 함수 정의 완료 (run_abnumber v4 적용).


In [ ]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------
# 3. 데이터 소스 처리: ABSD (쌍체 서열) - 경로 수정됨 v2
# -----------------------------------------------------------
import tarfile
import gzip
import shutil
from pathlib import Path
from Bio import SeqIO
import os
import logging
from tqdm.auto import tqdm
import time

# --- 상수 정의 (셀 1에서 가져옴 가정) ---
# ABSD_RAW_DIR, SUBSET_SIZE

def process_absd(metadata_list):
    """ABSD 데이터를 처리하여 메타데이터 리스트에 추가합니다. (경로 수정됨 v2)"""
    logging.info("="*30)
    logging.info(" ABSD 데이터 처리 시작 ")
    logging.info("="*30)

    absd_tar_path = os.path.join(ABSD_RAW_DIR, "absd_all_species.tar.gz")
    absd_extract_path = os.path.join(ABSD_RAW_DIR, "extracted")
    absd_data_subdir = os.path.join(absd_extract_path, "data") # data 하위 폴더 경로

    # 벌크 파일 존재 확인
    if not os.path.exists(absd_tar_path):
        logging.error(f"ABSD 벌크 파일({absd_tar_path})을 찾을 수 없습니다. 직접 다운로드 후 위치시켜 주세요. ABSD 처리 건너<0xEB><0x9B><0x84>니다.")
        return
    else:
        logging.info(f"기존 ABSD 벌크 파일 사용: {absd_tar_path}")

    # 압축 해제 및 인간 FASTA 경로 확인
    human_fasta_expected_name = "Homo_sapiens.fasta"
    human_fasta_path = os.path.join(absd_data_subdir, human_fasta_expected_name)

    if not os.path.exists(human_fasta_path):
        logging.info(f"예상 경로에 인간 FASTA 없음. '{absd_tar_path}' 압축 해제 시도...")
        try:
            should_extract = True
            if os.path.exists(absd_extract_path):
                 if os.path.exists(absd_data_subdir) and os.listdir(absd_data_subdir):
                      logging.info(f"'{absd_data_subdir}' 디렉토리가 이미 존재하고 내용이 있습니다. 압축 해제 건너<0xEB><0x9B><0x84>니다.")
                      should_extract = False
                 elif not os.path.exists(absd_data_subdir) and os.listdir(absd_extract_path):
                      logging.warning(f"'{absd_extract_path}'는 존재하나 'data' 하위 디렉토리가 없습니다. 압축 해제 시도.")
                 elif not os.listdir(absd_extract_path):
                      logging.info(f"'{absd_extract_path}' 디렉토리가 비어있습니다. 압축 해제 시도.")

            if should_extract:
                os.makedirs(absd_extract_path, exist_ok=True)
                with tarfile.open(absd_tar_path, "r:gz") as tar:
                    members_to_extract = [m for m in tar.getmembers() if m.isfile() and m.name.lower().endswith('.fasta')]
                    if not members_to_extract:
                        logging.error("ABSD 아카이브 내 FASTA 파일 없음."); return
                    logging.info(f"{len(members_to_extract)}개의 FASTA 파일을 추출합니다.")
                    for member in tqdm(members_to_extract, desc="ABSD FASTA 추출 중"):
                        try: tar.extract(member, path=absd_extract_path)
                        except Exception as e: logging.warning(f"{member.name} 추출 중 오류: {e}")
                logging.info(f"압축 해제 완료: {absd_extract_path}")
            else: # should_extract == False
                 pass # 이미 압축 해제된 것으로 간주

        except Exception as e:
            logging.error(f"ABSD 아카이브 압축 해제 실패: {e}"); return

        # 압축 해제 후 다시 확인
        if not os.path.exists(human_fasta_path):
             logging.error(f"압축 해제 후에도 예상 경로({human_fasta_path})에서 인간 FASTA 파일 찾기 실패.")
             alt_path = os.path.join(absd_extract_path, human_fasta_expected_name)
             if os.path.exists(alt_path):
                 human_fasta_path = alt_path
                 logging.warning(f"대체 경로에서 파일 발견: {human_fasta_path}")
             else:
                  logging.error(f"대체 경로({alt_path})에서도 파일을 찾을 수 없습니다. ABSD 처리 중단."); return
        else: logging.info(f"인간 FASTA 파일 확인: {human_fasta_path}")
    else: logging.info(f"기존 추출된 인간 FASTA 파일 사용: {human_fasta_path}")

    # FASTA 파싱
    logging.info(f"'{human_fasta_path}' 파싱 시작...")
    processed_count = 0
    try:
        parser = SeqIO.parse(human_fasta_path, "fasta")
        for record in tqdm(parser, desc="ABSD FASTA 파싱 중"):
            if SUBSET_SIZE and processed_count >= SUBSET_SIZE: break
            header = record.description; parts = header.split('_')
            if len(parts) >= 4:
                chain_type = parts[0]; species = parts[1] + "_" + parts[2]; absd_id = parts[3]
                entry = {
                    'entry_id': f"ABSD_{absd_id}_{chain_type}", 'source_database': 'ABSD', 'source_id': absd_id,
                    'species': species, 'chain_type': chain_type,
                    'vh_sequence': str(record.seq).upper() if chain_type == 'VH' else None,
                    'vl_sequence': str(record.seq).upper() if chain_type == 'VL' else None,
                    'header': header, 'pdb_id': None, 'h_chain_id': None, 'l_chain_id': None,
                }
                metadata_list.append(entry); processed_count += 1
        logging.info(f"ABSD 파싱 완료. {processed_count}개 서열 항목 처리.")
    except FileNotFoundError: logging.error(f"ABSD 인간 FASTA 파일({human_fasta_path})을 찾을 수 없습니다.")
    except Exception as e: logging.error(f"ABSD FASTA 파싱 중 오류: {e}")
    logging.info("ABSD 데이터 처리 완료.")

In [ ]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------
# 4. 데이터 소스 처리: SAbDab / PDB (구조-서열-친화도 연계) - 수정됨 v2
# -----------------------------------------------------------
import pandas as pd
import os
import logging
import asyncio
from tqdm.auto import tqdm
from pathlib import Path

# --- Helper Function 임포트 가정 ---
# from helper_functions import parse_sabdab_summary, download_pdb_files_async, extract_vh_vl_sequences_from_pdb

# --- 상수 정의 (셀 1에서 가져옴 가정) ---
# SABDAB_RAW_DIR, PDB_RAW_DIR, SABDAB_SUMMARY_URL, PDB_DOWNLOAD_URL, MIN_RESOLUTION, SUBSET_SIZE
# MIN_VH_LEN, MAX_VH_LEN, MIN_VL_LEN, MAX_VL_LEN

def parse_sabdab_summary(summary_file):
    """SAbDab 요약 TSV 파일을 파싱하여 DataFrame으로 반환합니다."""
    logging.info(f"SAbDab 요약 파일 로딩 중: {summary_file}")
    try:
        df = pd.read_csv(summary_file, sep='\t', low_memory=False, on_bad_lines='warn')
        df.columns = df.columns.str.lower().str.replace(' ', '_')
        logging.info(f"SAbDab 요약 파일 로드 완료. {len(df)}개 항목.")
        logging.debug(f"SAbDab 감지된 컬럼: {list(df.columns)}")
        essential_cols = ['pdb', 'hchain', 'lchain']
        missing_cols = [col for col in essential_cols if col not in df.columns]
        if missing_cols:
            logging.error(f"SAbDab 요약 필수 컬럼 누락: {missing_cols}. 실제 컬럼: {list(df.columns)}")
            return pd.DataFrame()
        return df
    except FileNotFoundError: logging.error(f"SAbDab 요약 파일({summary_file}) 없음."); return pd.DataFrame()
    except Exception as e: logging.error(f"SAbDab 요약 파일 처리 오류: {e}"); return pd.DataFrame()

async def download_pdb_files_async(pdb_ids_to_download, dest_dir):
    """주어진 PDB ID 목록에 대해 비동기적으로 PDB 파일을 다운로드합니다."""
    logging.info(f"{len(pdb_ids_to_download)}개의 PDB 파일 다운로드 시도...")
    urls_and_paths = []
    ids_to_check_existence = []
    for pdb_id in pdb_ids_to_download:
        if isinstance(pdb_id, str) and len(pdb_id) == 4:
             pdb_id_upper = pdb_id.upper(); pdb_id_lower = pdb_id.lower()
             dest_path = os.path.join(dest_dir, f"{pdb_id_lower}.pdb")
             ids_to_check_existence.append(pdb_id_lower) # 존재 여부 확인할 ID 목록
             if not os.path.exists(dest_path):
                 urls_and_paths.append((PDB_DOWNLOAD_URL.format(pdb_id_upper), dest_path))
        else: logging.warning(f"잘못된 PDB ID 형식 건너<0xEB><0x9B><0x84>: {pdb_id}")

    downloaded_results = await download_files_in_batch(urls_and_paths, desc="PDB 다운로드", concurrency=DOWNLOAD_CONCURRENCY)

    # 다운로드 결과와 기존 파일 존재 여부 통합하여 최종 사용 가능 ID 집합 생성
    successfully_available_pdbs = set()
    # 다운로드 성공한 파일 ID 추가
    successfully_available_pdbs.update(
        Path(p).stem for p, res in zip([url_path[1] for url_path in urls_and_paths], downloaded_results) if res is not None
    )
    # 원래 존재했던 파일 ID 추가
    for pdb_id_lower in ids_to_check_existence:
        if os.path.exists(os.path.join(dest_dir, f"{pdb_id_lower}.pdb")):
             successfully_available_pdbs.add(pdb_id_lower)

    downloaded_count = sum(1 for res in downloaded_results if res is not None)
    existing_count = len(successfully_available_pdbs) - downloaded_count
    failed_count = len(urls_and_paths) - downloaded_count

    logging.info(f"PDB 다운로드 완료. 신규 다운로드: {downloaded_count}, 기존 파일: {existing_count}, 실패: {failed_count}")
    return successfully_available_pdbs


async def process_sabdab(metadata_list):
    """SAbDab 데이터를 다운로드, 파싱하고 관련 PDB를 다운로드하여 메타데이터에 추가합니다."""
    logging.info("="*30)
    logging.info(" SAbDab / PDB 데이터 처리 시작 ")
    logging.info("="*30)

    sabdab_summary_file = os.path.join(SABDAB_RAW_DIR, "sabdab_summary.tsv")

    # 1. SAbDab 요약 파일 다운로드 (없으면)
    if not os.path.exists(sabdab_summary_file):
        logging.info(f"SAbDab 요약 파일 다운로드 시도: {SABDAB_SUMMARY_URL}")
        download_success = download_file_sync(SABDAB_SUMMARY_URL, sabdab_summary_file, desc="SAbDab 요약 다운로드")
        if not download_success: logging.error("SAbDab 요약 파일 다운로드 실패."); return
    else: logging.info(f"기존 SAbDab 요약 파일 사용: {sabdab_summary_file}")

    # 2. 요약 파일 파싱
    sabdab_df = parse_sabdab_summary(sabdab_summary_file)
    if sabdab_df.empty: logging.error("SAbDab 요약 파일 처리 실패."); return

    # 3. 데이터 필터링
    logging.info(f"초기 SAbDab 항목 수: {len(sabdab_df)}")
    initial_len = len(sabdab_df)
    # 해상도 필터
    if 'resolution' in sabdab_df.columns:
        sabdab_df['resolution'] = pd.to_numeric(sabdab_df['resolution'], errors='coerce')
        sabdab_df = sabdab_df[sabdab_df['resolution'] <= MIN_RESOLUTION].copy()
        logging.info(f"해상도 (<={MIN_RESOLUTION}Å) 필터링 후 항목 수: {len(sabdab_df)}")
    # 필수 컬럼 누락 행 제거
    sabdab_df.dropna(subset=['pdb', 'hchain', 'lchain'], inplace=True)
    logging.info(f"필수 ID 누락 제거 후 항목 수: {len(sabdab_df)}")

    if sabdab_df.empty: logging.warning("필터링 후 남은 SAbDab 항목이 없습니다."); return

    # 4. 처리할 PDB ID 목록 결정
    all_pdb_ids = sabdab_df['pdb'].unique()
    if SUBSET_SIZE and len(all_pdb_ids) > SUBSET_SIZE:
        pdb_ids_to_process = all_pdb_ids[:SUBSET_SIZE]
        logging.warning(f"SUBSET_SIZE 적용: {len(pdb_ids_to_process)}개의 PDB ID만 처리합니다.")
    else:
        pdb_ids_to_process = all_pdb_ids

    # 5. 관련 PDB 파일 다운로드
    available_pdb_ids = await download_pdb_files_async(pdb_ids_to_process, PDB_RAW_DIR)
    logging.info(f"사용 가능한 PDB 파일 {len(available_pdb_ids)}개 확인 (ID 기준).")

    # 6. SAbDab 항목 순회 및 메타데이터 추출
    logging.info("SAbDab 항목 처리 및 메타데이터 생성 시작...")
    processed_count = 0
    sabdab_df_filtered = sabdab_df[sabdab_df['pdb'].str.lower().isin(available_pdb_ids)].copy() # 사용 가능한 PDB만 필터링
    for index, row in tqdm(sabdab_df_filtered.iterrows(), total=len(sabdab_df_filtered), desc="SAbDab 항목 처리 중"):
        pdb_id = str(row['pdb']).lower()
        h_chain = str(row['hchain']).strip() if pd.notna(row['hchain']) else None
        l_chain = str(row['lchain']).strip() if pd.notna(row['lchain']) else None

        if not h_chain or not l_chain: continue
        pdb_file_path = os.path.join(PDB_RAW_DIR, f"{pdb_id}.pdb")
        if not os.path.exists(pdb_file_path): continue # 혹시 모를 재확인

        vh_seq, vl_seq = extract_vh_vl_sequences_from_pdb(pdb_file_path, h_chain, l_chain)
        if not vh_seq or not vl_seq: continue

        # 품질 필터링 (길이만 - 비표준 아미노산은 통합 단계에서)
        if not (MIN_VH_LEN <= len(vh_seq) <= MAX_VH_LEN and MIN_VL_LEN <= len(vl_seq) <= MAX_VL_LEN): continue

        # 메타데이터 구성
        entry = {
            'entry_id': f"SAbDab_{pdb_id}_{h_chain}_{l_chain}", 'source_database': 'SAbDab', 'source_id': pdb_id.upper(),
            'pdb_id': pdb_id.upper(), 'h_chain_id': h_chain, 'l_chain_id': l_chain,
            'vh_sequence': vh_seq, 'vl_sequence': vl_seq, 'raw_pdb_path': pdb_file_path,
            'antigen_chain': row.get('antigen_chain'), 'antigen_type': row.get('antigen_type'),
            'antigen_het_name': row.get('antigen_het_name'), 'antigen_name': row.get('antigen_name'),
            'heavy_species': row.get('heavy_species'), 'light_species': row.get('light_species'),
            'antigen_species': row.get('antigen_species'),
            'resolution': row.get('resolution'), # 이미 numeric으로 변환됨
            'affinity': pd.to_numeric(row.get('affinity'), errors='coerce'),
            'affinity_method': row.get('affinity_method'), 'temperature': pd.to_numeric(row.get('temperature'), errors='coerce'),
            'pmid': row.get('pmid'), 'exp_method': row.get('method'), 'deposition_date': row.get('date'),
            'structure_header': row.get('short_header'), 'compound_info': row.get('compound'),
            'scfv': row.get('scfv'), 'engineered': row.get('engineered'),
        }
        metadata_list.append(entry)
        processed_count += 1

    logging.info(f"SAbDab/PDB 처리 완료. {processed_count}개 유효 항목 메타데이터 생성.")

In [ ]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------
# 5. 데이터 소스 처리: AACDB - 최종 완성 버전 v4 (디버깅 로그 + PDB 경로 수정 v2)
# -----------------------------------------------------------
import pandas as pd
import os
import logging
from tqdm.auto import tqdm
from Bio import SeqIO
from collections import defaultdict
from pathlib import Path

# --- Helper Function 임포트 가정 ---
# from helper_functions import extract_vh_vl_sequences_from_pdb, parse_single_sasa_file, parse_single_distance_file

# --- 상수 임포트 가정 ---
# from config import AACDB_RAW_DIR, AACDB_SUMMARY_FILE, AACDB_ANTIBODY_FASTA, AACDB_PDB_DIR, ...
# --- MIN/MAX Length, MIN_RESOLUTION, STANDARD_AMINO_ACIDS 정의 가정 ---

def process_aacdb(metadata_list): # 인자에서 상호작용 데이터 제거
    """AACDB 데이터를 처리하고 메타데이터 리스트를 반환합니다. (PDB 경로 수정 v2 + Interaction 파싱 내장)"""
    logging.info("="*30)
    logging.info(" AACDB 데이터 처리 시작 (v5 - Interaction Parsing 내장) ")
    logging.info("="*30)

    aacdb_metadata = []
    processed_ids = set()

    # 파일 존재 확인 등 기본 설정
    required_files = [AACDB_SUMMARY_FILE, AACDB_ANTIBODY_FASTA]
    missing_files = [f for f in required_files if not os.path.exists(f)]
    if missing_files: logging.error(f"필수 AACDB 파일 누락: {missing_files}. 처리 건너<0xEB><0x9B><0x84>니다."); return
    logging.info("필요한 AACDB 파일 존재 확인됨.")

    # --- 정확한 PDB 경로 사용 ---
    aacdb_pdb_base_dir = AACDB_PDB_DIR # 셀 1에서 정의된 상수 사용
    logging.info(f"주 PDB 파일 경로로 사용: {aacdb_pdb_base_dir}")
    if not os.path.isdir(aacdb_pdb_base_dir): logging.error(f"AACDB PDB 디렉토리({aacdb_pdb_base_dir}) 없음."); return

    # --- 상호작용 데이터 파싱 (여기서 수행) ---
    logging.info("AACDB 상호작용 데이터 파싱 시작...")
    interaction_sasa_parsed = {}
    interaction_dist_parsed = {}
    sasa_files_found = list(Path(AACDB_INTERACTION_SASA_DIR).glob('*.txt'))
    dist_files_found = list(Path(AACDB_INTERACTION_DIST_DIR).glob('*.*')) # 확장자 다양 가능성

    if sasa_files_found:
        for file_path in tqdm(sasa_files_found, desc="Parsing SASA files"):
            pdb_id = file_path.stem.split('_')[0].lower() if '_' in file_path.stem else file_path.stem.lower()
            if len(pdb_id) == 4:
                sasa_data = parse_single_sasa_file(str(file_path))
                if sasa_data: interaction_sasa_parsed[pdb_id] = sasa_data
    else: logging.warning(f"SASA 상호작용 파일 없음: {AACDB_INTERACTION_SASA_DIR}")

    if dist_files_found:
        for file_path in tqdm(dist_files_found, desc="Parsing Distance files"):
            if not file_path.is_file(): continue
            pdb_id = file_path.stem.split('_')[0].lower() if '_' in file_path.stem else file_path.stem.lower()
            if len(pdb_id) == 4:
                dist_data = parse_single_distance_file(str(file_path))
                if dist_data: interaction_dist_parsed[pdb_id] = dist_data
    else: logging.warning(f"Distance 상호작용 파일 없음: {AACDB_INTERACTION_DIST_DIR}")
    logging.info(f"파싱된 SASA 항목 수: {len(interaction_sasa_parsed)}, Distance 항목 수: {len(interaction_dist_parsed)}")
    # --- 상호작용 데이터 파싱 끝 ---

    # 요약 파일 파싱
    logging.info(f"AACDB 요약 파일 파싱 시작: {AACDB_SUMMARY_FILE}")
    try:
        aacdb_summary_df = pd.read_csv(AACDB_SUMMARY_FILE, sep='\t', comment='#', low_memory=False)
        logging.info(f"AACDB 요약 로드 완료: {len(aacdb_summary_df)} 항목. 컬럼: {list(aacdb_summary_df.columns)}")
    except Exception as e: logging.error(f"AACDB 요약 파일 로딩/파싱 실패: {e}"); return

    # FASTA 파일 파싱
    logging.info(f"AACDB 항체 FASTA 파싱 시작: {AACDB_ANTIBODY_FASTA}")
    antibody_seqs = {}
    try:
        for record in SeqIO.parse(AACDB_ANTIBODY_FASTA, "fasta"):
            parts = record.id.split('|')[0].split('_')
            if len(parts) >= 3:
                pdb_id = parts[0]; chain_id = parts[2]
                entry_key = f"{pdb_id.upper()}_{chain_id}"
                antibody_seqs[entry_key] = str(record.seq).upper()
            else: logging.warning(f"AACDB FASTA ID 형식 예상과 다름: {record.id}")
    except Exception as e: logging.error(f"AACDB FASTA 파일 파싱 실패: {e}")
    logging.info(f"AACDB 항체 FASTA 로드 완료: {len(antibody_seqs)}개 서열")

    # 메타데이터 생성 (FASTA 기반)
    logging.info("AACDB 항목 처리 및 메타데이터 생성 시작 (FASTA 기반)...")
    paired_entries = defaultdict(dict)
    for key, seq in tqdm(antibody_seqs.items(), desc="AACDB FASTA 쌍 찾는 중"):
        try:
             pdb_id, chain_id = key.split('_')
             if chain_id.endswith('H'): paired_entries[pdb_id]['VH'] = {'id': chain_id, 'seq': seq}
             elif chain_id.endswith('L'): paired_entries[pdb_id]['VL'] = {'id': chain_id, 'seq': seq}
        except ValueError: logging.warning(f"FASTA key 파싱 오류: {key}")

    pdb_not_found_count = 0
    pdb_files_in_dir_cache = None # PDB 파일 목록 캐싱

    for pdb_id_upper, chains in tqdm(paired_entries.items(), desc="AACDB 메타데이터 생성"):
        if 'VH' in chains and 'VL' in chains:
            vh_info = chains['VH']; vl_info = chains['VL']
            vh_chain_id = vh_info['id']; vl_chain_id = vl_info['id']
            vh_seq = vh_info['seq']; vl_seq = vl_info['seq']

            summary_row = aacdb_summary_df[aacdb_summary_df['pdb'].str.upper() == pdb_id_upper].iloc[0] if not aacdb_summary_df[aacdb_summary_df['pdb'].str.upper() == pdb_id_upper].empty else None

            # PDB 파일 경로 찾기 로직 수정 v2
            actual_pdb_path = None; pdb_id_lower = pdb_id_upper.lower()
            if pdb_files_in_dir_cache is None: # 최초 1회만 실행
                try: pdb_files_in_dir_cache = {f.lower(): f for f in os.listdir(aacdb_pdb_base_dir) if f.lower().endswith('.pdb')}
                except FileNotFoundError: logging.error(f"AACDB PDB 디렉토리 접근 불가: {aacdb_pdb_base_dir}"); pdb_files_in_dir_cache = {}

            found_filename_lower = next((fname for fname in pdb_files_in_dir_cache if fname.startswith(pdb_id_lower + "_")), None)

            if found_filename_lower:
                actual_filename = pdb_files_in_dir_cache.get(found_filename_lower) # 캐시에서 원래 이름 가져오기
                if actual_filename:
                    actual_pdb_path = os.path.join(aacdb_pdb_base_dir, actual_filename)
                    logging.debug(f"PDB 파일 매칭 성공: {pdb_id_upper} -> {actual_filename}")
                else: # 캐시에 소문자만 저장된 경우 (거의 없음)
                    actual_pdb_path = os.path.join(aacdb_pdb_base_dir, found_filename_lower)
                    logging.warning(f"원본 대소문자 파일명 복원 실패, 소문자 사용: {found_filename_lower}")
            else:
                pdb_not_found_count += 1
                logging.debug(f"PDB 파일 찾기 실패 (파일 목록 검색): {pdb_id_upper} in {aacdb_pdb_base_dir}")
                continue

            entry_id = f"AACDB_{pdb_id_upper}_{vh_chain_id}_{vl_chain_id}"
            if entry_id in processed_ids: continue

            resolution_raw = summary_row['resolution'] if summary_row is not None and 'resolution' in summary_row else None
            resolution = None
            if pd.notna(resolution_raw):
                 try: resolution = float(''.join(filter(lambda x: x.isdigit() or x == '.', str(resolution_raw))))
                 except ValueError: logging.warning(f"[AACDB Warning] PDB: {pdb_id_upper}, 해상도 파싱 오류: {resolution_raw}")

            # 필터링
            if pd.notna(resolution) and resolution > MIN_RESOLUTION: logging.debug(f"[AACDB FILTER] Skipping {pdb_id_upper} (Res)"); continue
            if not (MIN_VH_LEN <= len(vh_seq) <= MAX_VH_LEN and MIN_VL_LEN <= len(vl_seq) <= MAX_VL_LEN): logging.debug(f"[AACDB FILTER] Skipping {pdb_id_upper} (Len)"); continue

            # 메타데이터 구성 (interaction 정보는 파싱된 결과 사용)
            sasa_info = interaction_sasa_parsed.get(pdb_id_lower, {}) # 소문자 ID로 조회
            dist_info = interaction_dist_parsed.get(pdb_id_lower, {})

            meta_entry = {
                 "entry_id": entry_id, "source_database": "AACDB",
                 "source_id": summary_row['id'] if summary_row is not None and 'id' in summary_row else pdb_id_upper,
                 "pdb_id": pdb_id_upper, "h_chain_id": vh_chain_id, "l_chain_id": vl_chain_id,
                 "vh_sequence": vh_seq, "vl_sequence": vl_seq, "sequence_source": "AACDB_FASTA",
                 "resolution": resolution,
                 "exp_method": summary_row.get('method', None), # .get 사용
                 "affinity": None, # 필요 시 추가
                 "antibody_name": summary_row.get('antibody', None),
                 "antigen_name": summary_row.get('protein', None),
                 "antigen_targets": summary_row.get('targets', None),
                 "heavy_species": summary_row.get('organism', None),
                 "light_species": summary_row.get('organism', None),
                 "organism_antibody": summary_row.get('organism', None),
                 "organism_antigen": summary_row.get('organism', None),
                 "interaction_residues_sasa_ab": ",".join(sasa_info.get('antibody',[])) if sasa_info else None, # get 기본값 []
                 "interaction_residues_sasa_ag": ",".join(sasa_info.get('antigen',[])) if sasa_info else None,
                 "interaction_residues_dist_ab": ",".join(sorted(list(dist_info.get('antibody',set())))) if dist_info else None, # get 기본값 set()
                 "interaction_residues_dist_ag": ",".join(sorted(list(dist_info.get('antigen',set())))) if dist_info else None,
                 "reference_pubmed": summary_row.get('reference', None),
                 "raw_pdb_path": actual_pdb_path
             }
            metadata_list.append(meta_entry)
            processed_ids.add(entry_id)

    logging.info(f"AACDB 처리 완료. 최종 {len(aacdb_metadata)}개 유효 항목 메타데이터 생성.")
    logging.info(f"  - FASTA에서 찾은 VH/VL 쌍 수: {len(paired_entries)}")
    if pdb_not_found_count > 0:
         logging.warning(f"  - PDB 파일을 찾지 못한 항목 수: {pdb_not_found_count}")

    return aacdb_metadata

logging.info("process_aacdb 함수 정의 완료 (v5).")

2025-04-27 12:36:55,426 - INFO     - process_aacdb 함수 정의 완료 (v5).


In [ ]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------
# 6. 데이터 소스 처리: OAS (대규모 서열 레퍼토리 - CSV 처리 버전 v2)
# -----------------------------------------------------------
import gzip
from collections import defaultdict
import pandas as pd
import os
import logging
from tqdm.auto import tqdm
from pathlib import Path
import json
import shutil
import traceback

# --- 상수 정의 (셀 1에서 가져옴 가정) ---
# OAS_RAW_DIR, SUBSET_SIZE, MIN_VH_LEN, MAX_VH_LEN, MIN_VL_LEN, MAX_VL_LEN, STANDARD_AMINO_ACIDS

def process_oas(metadata_list):
    """
    로컬에 다운로드된 OAS CSV 데이터(.csv.gz)를 처리하여 VH/VL 쌍으로 구성하고 메타데이터 리스트에 추가합니다.
    사용자가 bulk_download.sh를 미리 실행했다고 가정합니다.
    """
    logging.info("="*30)
    logging.warning(" OAS 데이터 처리 시작 (v3.1: 로컬 CSV 파일 파싱 및 페어링) ")
    logging.info("="*30)
    logging.warning(f"OAS 데이터 디렉토리({OAS_RAW_DIR})에서 .csv.gz 파일을 검색합니다.")
    logging.warning("bulk_download.sh 스크립트가 이 디렉토리에 파일을 다운로드했어야 합니다.")

    processed_pairs_count = 0
    oas_files = list(Path(OAS_RAW_DIR).glob('*.csv.gz'))

    if not oas_files:
        logging.error(f"OAS 데이터 파일(.csv.gz)을 찾을 수 없습니다: {OAS_RAW_DIR}")
        return

    logging.info(f"{len(oas_files)}개의 OAS .csv.gz 파일 처리 시작...")

    vh_seq_col = 'sequence_alignment_aa_heavy'
    vl_seq_col = 'sequence_alignment_aa_light'
    vh_v_call_col = 'v_call_heavy'; vh_j_call_col = 'j_call_heavy'
    vl_v_call_col = 'v_call_light'; vl_j_call_col = 'j_call_light'
    vh_cdr3_col = 'cdr3_aa_heavy'; vl_cdr3_col = 'cdr3_aa_light'
    isotype_col = 'Isotype_heavy'

    for file_path in tqdm(oas_files, desc="Processing OAS files"):
        oas_run_id = file_path.stem.replace('_1_Paired_All.csv', '')
        if SUBSET_SIZE and processed_pairs_count >= SUBSET_SIZE: break

        try:
            # --- 수정: 파일 헤더 JSON 파싱 오류 방지 ---
            file_species = "Unknown"
            try:
                with gzip.open(file_path, 'rt', encoding='utf-8') as f:
                    first_line = f.readline().strip()
                    # JSON 파싱 시도하되, 실패해도 경고만 남기고 진행
                    if first_line.startswith('{') and first_line.endswith('}'):
                        try:
                            # 양 끝 따옴표 제거 시도 (일부 파일 문제 해결 가능성)
                            cleaned_line = first_line.strip('"')
                            meta_json = json.loads(cleaned_line)
                            file_species = meta_json.get("Species", "Unknown")
                        except json.JSONDecodeError as e_json:
                            logging.warning(f"OAS 파일 '{file_path.name}' 메타데이터 JSON 파싱 실패 (내용 확인 필요): {e_json}")
                    else:
                         logging.warning(f"OAS 파일 '{file_path.name}'의 첫 줄이 예상된 JSON 형식이 아님: {first_line[:100]}...")
            except Exception as e_head:
                 logging.warning(f"OAS 파일 '{file_path.name}'의 헤더 읽기/처리 중 오류: {e_head}")
            # --- 수정 끝 ---

            # CSV 읽기 (header=1 설정 유지)
            df = pd.read_csv(file_path, compression='gzip', header=1, low_memory=False, on_bad_lines='warn')
            logging.debug(f"Processing {file_path.name}, Species: {file_species}, Columns found: {len(df.columns)}")

            required_cols = [vh_seq_col, vl_seq_col]
            if not all(col in df.columns for col in required_cols):
                logging.warning(f"'{file_path.name}'에 서열 컬럼({required_cols}) 없음. 건너<0xEB><0x9B><0x84>니다.")
                continue

            # DataFrame 순회하며 메타데이터 생성
            for index, row in tqdm(df.iterrows(), total=len(df), desc=f"Parsing {oas_run_id}", leave=False):
                if SUBSET_SIZE and processed_pairs_count >= SUBSET_SIZE: break

                vh_seq = row.get(vh_seq_col); vl_seq = row.get(vl_seq_col)
                if not isinstance(vh_seq, str) or not isinstance(vl_seq, str): continue

                # 품질 필터링
                if (MIN_VH_LEN <= len(vh_seq) <= MAX_VH_LEN) and \
                   (MIN_VL_LEN <= len(vl_seq) <= MAX_VL_LEN) and \
                   all(aa in STANDARD_AMINO_ACIDS for aa in vh_seq) and \
                   all(aa in STANDARD_AMINO_ACIDS for aa in vl_seq):

                    entry_id = f"OAS_{oas_run_id}_{index}"
                    entry = { # 메타데이터 구성 (이전과 동일)
                        'entry_id': entry_id, 'source_database': 'OAS', 'source_id': f"{oas_run_id}_{index}",
                        'species': file_species, 'heavy_species': file_species, 'light_species': file_species,
                        'vh_sequence': vh_seq.upper(), 'vl_sequence': vl_seq.upper(),
                        'vh_cdr3': row.get(vh_cdr3_col), 'vl_cdr3': row.get(vl_cdr3_col),
                        'v_call_heavy': row.get(vh_v_call_col), 'j_call_heavy': row.get(vh_j_call_col),
                        'v_call_light': row.get(vl_v_call_col), 'j_call_light': row.get(vl_j_call_col),
                        'isotype_heavy': row.get(isotype_col),
                        'sequence_source': 'OAS_CSV',
                        'pdb_id': None, 'h_chain_id': 'H', 'l_chain_id': 'L',
                        'raw_pdb_path': None, 'resolution': None, 'affinity': None,
                    }
                    metadata_list.append(entry)
                    processed_pairs_count += 1
                else: logging.debug(f"OAS 항목 {entry_id} 필터링됨")

        except pd.errors.EmptyDataError: logging.warning(f"OAS 파일 비어있음: {file_path.name}")
        except KeyError as e: logging.warning(f"OAS 파일 '{file_path.name}' 컬럼 '{e}' 없음.")
        except Exception as e: logging.error(f"OAS 파일 ({file_path.name}) 처리 오류: {e}"); traceback.print_exc()

    logging.info(f"OAS 데이터 처리 완료. 최종 {processed_pairs_count}개 유효 쌍 항목 처리.")

logging.info("process_oas 함수 정의 완료 (CSV 처리 버전 v3.1).")

2025-04-27 12:36:55,447 - INFO     - process_oas 함수 정의 완료 (CSV 처리 버전 v3.1).


In [ ]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------
# 7. 데이터 통합 및 전처리 - 셀 7 전체 코드 (AbNumber + OAS 처리 반영됨)
# -----------------------------------------------------------
import pandas as pd
import os
import logging
from tqdm.auto import tqdm
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pathlib import Path
import shutil
from Bio.PDB import PDBParser, PDBIO, Select, Polypeptide

# --- Helper Function 임포트 가정 ---
# from helper_functions import run_abnumber, extract_vh_vl_sequences_from_pdb

# --- 상수 정의 (셀 1에서 가져옴 가정) ---
# STANDARD_AMINO_ACIDS, MIN_VH_LEN, MAX_VH_LEN, MIN_VL_LEN, MAX_VL_LEN
# FINAL_METADATA_FILE, FINAL_SEQS_DIR, FINAL_PDB_DIR

def preprocess_and_integrate_data(absd_meta, sabdab_meta, aacdb_meta, oas_meta):
    """
    수집된 모든 소스의 메타데이터를 통합하고 전처리(중복제거, 필터링, AbNumber 번호부여)를 수행합니다.
    (OAS 데이터 처리 로직 포함)
    """
    logging.info("="*30)
    logging.info(" 데이터 통합 및 전처리 시작 (AbNumber + OAS 사용) ")
    logging.info("="*30)

    # 7.1. 모든 메타데이터 통합
    logging.info("메타데이터 통합 중...")
    all_metadata_dfs = []
    if absd_meta: all_metadata_dfs.append(pd.DataFrame(absd_meta))
    if sabdab_meta: all_metadata_dfs.append(pd.DataFrame(sabdab_meta))
    if aacdb_meta: all_metadata_dfs.append(pd.DataFrame(aacdb_meta))
    if oas_meta: all_metadata_dfs.append(pd.DataFrame(oas_meta)) # OAS 데이터 추가

    if not all_metadata_dfs: logging.error("통합할 메타데이터 없음."); return None

    all_metadata = pd.concat(all_metadata_dfs, ignore_index=True)
    logging.info(f"통합 전 총 항목 수: {len(all_metadata)}")
    if all_metadata.empty: logging.error("통합된 메타데이터 없음."); return None

    # 7.2. 쌍(Paired) 데이터 구성 및 검증
    logging.info("VH/VL 쌍 데이터 구성 및 검증 중...")
    # ABSD/OAS 처리 함수에서 이미 VH/VL 컬럼을 채웠으므로, 바로 필터링 가능
    valid_pairs_df = all_metadata[
        (all_metadata['vh_sequence'].notna()) & (all_metadata['vh_sequence'] != '') &
        (all_metadata['vl_sequence'].notna()) & (all_metadata['vl_sequence'] != '')
    ].copy()
    logging.info(f"VH/VL 쌍이 확인된 항목 수: {len(valid_pairs_df)}")
    if valid_pairs_df.empty: logging.error("유효한 VH/VL 쌍 데이터 없음."); return None

    final_df = valid_pairs_df.reset_index(drop=True)

    # 7.3. 중복 제거 (VH+VL 서열 기준)
    logging.info("서열 기준 중복 제거 중...")
    initial_count = len(final_df)
    final_df.drop_duplicates(subset=['vh_sequence', 'vl_sequence'], keep='first', inplace=True)
    logging.info(f"중복 제거 후 항목 수: {len(final_df)} (제거된 중복: {initial_count - len(final_df)})")
    if final_df.empty: logging.warning("중복 제거 후 남은 데이터 없음."); return None

    # 7.4. 기본 품질 필터링
    logging.info("서열 길이 및 비표준 아미노산 필터링 중...")
    initial_count = len(final_df)
    try: standard_aa_set = set(STANDARD_AMINO_ACIDS)
    except NameError: logging.error("STANDARD_AMINO_ACIDS 상수 미정의."); return None

    valid_indices = []
    for index, row in tqdm(final_df.iterrows(), total=len(final_df), desc="품질 필터링"):
        vh_seq = row.get('vh_sequence', ''); vl_seq = row.get('vl_sequence', '')
        valid = True
        if not (MIN_VH_LEN <= len(vh_seq) <= MAX_VH_LEN and MIN_VL_LEN <= len(vl_seq) <= MAX_VL_LEN): valid = False
        if any(aa not in standard_aa_set for aa in vh_seq) or any(aa not in standard_aa_set for aa in vl_seq): valid = False
        if valid: valid_indices.append(index)

    final_df = final_df.loc[valid_indices].copy()
    logging.info(f"기본 품질 필터링 후 항목 수: {len(final_df)} (제거: {initial_count - len(final_df)})")
    if final_df.empty: logging.warning("품질 필터링 후 남은 데이터 없음."); # 계속 진행

    # 7.5. IMGT 번호 부여 (AbNumber 사용)
    logging.info("IMGT 번호 부여 및 CDR 추출 시작 (AbNumber 사용)...")
    cdr_sequences_vh, cdr_sequences_vl = {}, {}
    abnumber_status_vh, abnumber_status_vl = {}, {}
    sequences_to_process = {}
    for index, row in final_df.iterrows():
        vh_key = f"vh_{index}"; vl_key = f"vl_{index}"
        if pd.notna(row.get('vh_sequence')): sequences_to_process.setdefault(row['vh_sequence'], []).append(vh_key)
        if pd.notna(row.get('vl_sequence')): sequences_to_process.setdefault(row['vl_sequence'], []).append(vl_key)

    logging.info(f"AbNumber 처리 대상 고유 서열 수: {len(sequences_to_process)}")
    try:
        for seq, keys in tqdm(sequences_to_process.items(), desc="AbNumber 실행 중"):
            numbered_residues, cdr_sequences, status = run_abnumber(seq) # run_abnumber 함수 사용
            status_msg = status if status else "Failed"
            for key in keys:
                 is_vh = key.startswith('vh_')
                 if is_vh: cdr_sequences_vh[key] = cdr_sequences if cdr_sequences else {}; abnumber_status_vh[key] = status_msg
                 else: cdr_sequences_vl[key] = cdr_sequences if cdr_sequences else {}; abnumber_status_vl[key] = status_msg
    except NameError: logging.error("run_abnumber 함수 미정의."); return None
    except Exception as e: logging.error(f"AbNumber 실행 오류: {e}")

    logging.info("Adding AbNumber result columns to DataFrame...")
    try:
        final_df['abnumber_status_vh'] = final_df.index.map(lambda idx: abnumber_status_vh.get(f"vh_{idx}", "NotProcessed"))
        final_df['abnumber_status_vl'] = final_df.index.map(lambda idx: abnumber_status_vl.get(f"vl_{idx}", "NotProcessed"))
        final_df['vh_cdr1'] = final_df.index.map(lambda idx: cdr_sequences_vh.get(f"vh_{idx}", {}).get('CDR1'))
        final_df['vh_cdr2'] = final_df.index.map(lambda idx: cdr_sequences_vh.get(f"vh_{idx}", {}).get('CDR2'))
        final_df['vh_cdr3'] = final_df.index.map(lambda idx: cdr_sequences_vh.get(f"vh_{idx}", {}).get('CDR3'))
        final_df['vl_cdr1'] = final_df.index.map(lambda idx: cdr_sequences_vl.get(f"vl_{idx}", {}).get('CDR1'))
        final_df['vl_cdr2'] = final_df.index.map(lambda idx: cdr_sequences_vl.get(f"vl_{idx}", {}).get('CDR2'))
        final_df['vl_cdr3'] = final_df.index.map(lambda idx: cdr_sequences_vl.get(f"vl_{idx}", {}).get('CDR3'))
        logging.info("AbNumber columns added successfully.")
    except Exception as e: logging.error(f"Failed to add AbNumber result columns: {e}")

    logging.info(f"Columns after attempting to add AbNumber status: {list(final_df.columns)}") # 컬럼 목록 로깅 추가
    if 'abnumber_status_vh' in final_df.columns: logging.info(f"AbNumber VH Status Counts:\n{final_df['abnumber_status_vh'].value_counts().to_string()}")
    if 'abnumber_status_vl' in final_df.columns: logging.info(f"AbNumber VL Status Counts:\n{final_df['abnumber_status_vl'].value_counts().to_string()}")

    # AbNumber 성공 필터링
    logging.info("AbNumber 결과 필터링 시작...")
    initial_count = len(final_df)
    if 'abnumber_status_vh' in final_df.columns and 'abnumber_status_vl' in final_df.columns:
        final_df = final_df[
             final_df['abnumber_status_vh'].fillna('Failed').str.startswith('Success') &
             final_df['abnumber_status_vl'].fillna('Failed').str.startswith('Success')
        ].copy()
        logging.info(f"AbNumber 성공 항목 필터링 후: {len(final_df)} (제거: {initial_count - len(final_df)})")
    else: logging.warning("AbNumber 상태 컬럼 부재로 필터링 건너<0xEB><0x9B><0x84>.")

    # AbNumber 처리 후 서열 기준 중복 제거
    logging.info("AbNumber 처리 후 서열 기준 중복 제거 중...")
    initial_count = len(final_df)
    final_df.drop_duplicates(subset=['vh_sequence', 'vl_sequence'], keep='first', inplace=True)
    logging.info(f"최종 중복 제거 후 항목 수: {len(final_df)} (제거된 중복: {initial_count - len(final_df)})")

    if final_df.empty: logging.warning("최종 데이터셋이 비어있습니다."); # 계속 진행

    # 7.6. 최종 데이터 저장
    logging.info("최종 데이터 저장 중...")
    try:
        # 최종 컬럼 순서 정의 (모든 소스 가능성 포함)
        final_columns_order = [
            'entry_id', 'source_database', 'source_id', 'pdb_id', 'h_chain_id', 'l_chain_id',
            'vh_sequence', 'vl_sequence', 'vh_cdr1', 'vh_cdr2', 'vh_cdr3',
            'vl_cdr1', 'vl_cdr2', 'vl_cdr3', 'sequence_source', 'chain_type', # OAS/ABSD 용
            'species', 'heavy_species', 'light_species', 'organism_antibody', 'organism_antigen', 'antigen_species',
            'resolution', 'exp_method', 'affinity', 'affinity_method', 'temperature',
            'antibody_name', 'antigen_name', 'antigen_targets',
            'interaction_residues_sasa_ab', 'interaction_residues_sasa_ag',
            'interaction_residues_dist_ab', 'interaction_residues_dist_ag',
            'reference_pubmed', 'pmid', 'deposition_date', 'structure_header', 'compound_info',
            'scfv', 'engineered', 'header', 'header_vh', 'header_vl', # OAS/ABSD 용
            'raw_pdb_path', 'abnumber_status_vh', 'abnumber_status_vl',
            'v_call_heavy', 'j_call_heavy', 'v_call_light', 'j_call_light', 'isotype_heavy' # OAS에서 추가된 정보
        ]
        existing_columns = [col for col in final_columns_order if col in final_df.columns]
        final_df_to_save = final_df[existing_columns]

        os.makedirs(os.path.dirname(FINAL_METADATA_FILE), exist_ok=True)
        final_df_to_save.to_parquet(FINAL_METADATA_FILE, index=False)
        logging.info(f"최종 메타데이터 저장 완료: {FINAL_METADATA_FILE} ({len(final_df_to_save)} 항목)")
    except Exception as e:
        logging.error(f"최종 메타데이터 Parquet 저장 실패: {e}")
        alt_csv_path = os.path.splitext(FINAL_METADATA_FILE)[0] + ".csv"
        try: final_df_to_save.to_csv(alt_csv_path, index=False, encoding='utf-8-sig')
        except Exception as e_csv: logging.error(f"CSV 대체 저장 실패: {e_csv}")

    # 서열 저장 (FASTA)
    logging.info("최종 서열 FASTA 파일 저장 중...")
    os.makedirs(FINAL_SEQS_DIR, exist_ok=True)
    saved_pair_count = 0
    try:
        fasta_out_path = os.path.join(FINAL_SEQS_DIR, "final_antibody_seqs_abnumber.fasta")
        with open(fasta_out_path, "w") as f_out:
            if not final_df.empty:
                 for index, row in tqdm(final_df.iterrows(), total=len(final_df), desc="FASTA 저장"):
                    vh_id = f"{row.get('entry_id', f'idx{index}')}_VH H:{row.get('h_chain_id','NA')} L:{row.get('l_chain_id','NA')} PDB:{row.get('pdb_id','NA')}"
                    f_out.write(f">{vh_id}\n{row['vh_sequence']}\n")
                    vl_id = f"{row.get('entry_id', f'idx{index}')}_VL H:{row.get('h_chain_id','NA')} L:{row.get('l_chain_id','NA')} PDB:{row.get('pdb_id','NA')}"
                    f_out.write(f">{vl_id}\n{row['vl_sequence']}\n")
                    saved_pair_count += 1
        logging.info(f"{saved_pair_count}개 항체 쌍 (VH/VL) 서열 저장 완료: {fasta_out_path}")
    except Exception as e: logging.error(f"FASTA 파일 저장 중 오류: {e}")

    # 관련 PDB 파일 정리 (OAS/ABSD 제외)
    logging.info("관련 PDB 구조 파일 정리 중...")
    os.makedirs(FINAL_PDB_DIR, exist_ok=True)
    copied_pdb_count = 0
    pdb_parser = PDBParser(QUIET=True)
    pdb_io = PDBIO()

    class AntibodySelect(Select):
        def __init__(self, h_chain_id, l_chain_id): self.h_chain_id = h_chain_id; self.l_chain_id = l_chain_id
        def accept_chain(self, chain): return chain.id == self.h_chain_id or chain.id == self.l_chain_id
        def accept_residue(self, residue): return residue.id[0] == ' ' and Polypeptide.is_aa(residue.get_resname(), standard=True)

    if not final_df.empty:
        # PDB 파일이 있는 소스(SAbDab, AACDB)만 처리
        df_with_pdb = final_df[final_df['raw_pdb_path'].notna()].copy()
        for index, row in tqdm(df_with_pdb.iterrows(), total=len(df_with_pdb), desc="PDB 파일 정리"):
            raw_path = row['raw_pdb_path']; h_chain = row['h_chain_id']; l_chain = row['l_chain_id']
            pdb_id = str(row.get('pdb_id','unknown')).lower() # 소문자 ID 사용
            if os.path.exists(raw_path):
                final_pdb_path = os.path.join(FINAL_PDB_DIR, f"{pdb_id}_{h_chain}_{l_chain}.pdb")
                try:
                    struct = pdb_parser.get_structure(pdb_id, raw_path)
                    pdb_io.set_structure(struct)
                    pdb_io.save(final_pdb_path, AntibodySelect(h_chain, l_chain))
                    copied_pdb_count += 1
                except Exception as e:
                     logging.warning(f"{pdb_id} PDB 파일 처리/저장 실패: {e}. 원본 복사 시도.")
                     try: shutil.copyfile(raw_path, final_pdb_path); copied_pdb_count += 1
                     except Exception as e_copy: logging.error(f"{pdb_id} 원본 PDB 파일 복사 실패: {e_copy}")

    logging.info(f"{copied_pdb_count}개의 관련 PDB 파일 저장/복사 완료: {FINAL_PDB_DIR}")

    logging.info("데이터 통합 및 전처리 완료.")
    return final_df

In [ ]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------
# 8. 특성 공학 (Feature Engineering) - 기본 및 향후 방향
# -----------------------------------------------------------
from Bio.PDB import Polypeptide # Polypeptide 임포트 확인
import logging

def generate_basic_features(final_metadata_df):
    """기본적인 특성(예: One-Hot Encoding)을 생성합니다."""
    logging.info("="*30)
    logging.info(" 기본 특성 공학 시작 (One-Hot Encoding 예시) ")
    logging.info("="*30)

    if final_metadata_df is None or final_metadata_df.empty:
        logging.warning("메타데이터가 없어 특성 공학을 건너<0xEB><0x9B><0x84>니다.")
        return

    # --- One-Hot Encoding 예시 ---
    logging.info("One-Hot Encoding (간단 예시 - 실제 사용 시 보완 필요)")
    # STANDARD_AMINO_ACIDS가 셀 1에서 정의되었다고 가정
    try:
        aa_vocab = STANDARD_AMINO_ACIDS + 'X' # 표준 + 미지('X')
    except NameError:
        logging.error("STANDARD_AMINO_ACIDS 상수가 정의되지 않았습니다. 셀 1을 확인하세요.")
        aa_vocab = "ACDEFGHIKLMNPQRSTVWYX" # Fallback

    aa_to_int = {aa: i for i, aa in enumerate(aa_vocab)}

    def one_hot_encode(seq, max_len=150):
        encoded = []
        seq_pad = seq[:max_len].ljust(max_len, '-')
        for aa in seq_pad:
            vec = [0] * len(aa_vocab)
            aa_idx = aa_to_int.get(aa, aa_to_int.get('X'))
            if aa != '-': vec[aa_idx] = 1
            encoded.append(vec)
        return encoded

    # 예시 출력 (실제 인코딩/저장은 2단계에서 수행)
    logging.warning("One-Hot Encoding은 기본적이며, 서열/구조적 문맥 정보를 반영하지 못합니다.")

    # --- 향후 특성 공학 방향 ---
    logging.info("="*30); logging.info(" 향후 특성 공학 방향 (2단계 수행 권장) "); logging.info("="*30)
    logging.info("1. 단백질 언어 모델 (pLM) 임베딩: Hugging Face transformers (ESM, ProtBERT, AntiBERTy 등)...")
    logging.info("2. 구조 기반 특성 (GNN 입력용): BioPython, PyTorch Geometric (PyG)...")
    logging.info("="*30)

In [ ]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------
# 9. 데이터 분할 전략 (구현 가이드)
# -----------------------------------------------------------
import logging
import pandas as pd # pandas 사용 시 필요
from sklearn.model_selection import GroupShuffleSplit # 예시용

def guide_data_splitting(final_metadata_df):
    """데이터 분할 전략을 설명하고 구현 예시를 제공합니다."""
    logging.info("="*50)
    logging.info(" 신뢰성 있는 모델 검증을 위한 데이터 분할 전략 ")
    logging.info("="*50)

    if final_metadata_df is None or final_metadata_df.empty:
        logging.warning("메타데이터가 없어 데이터 분할 가이드를 건너<0xEB><0x9B><0x84>니다.")
        return

    logging.info("배경: 단순 무작위 분할은 항체 데이터의 높은 유사성 때문에 모델 성능을 과대평가할 수 있습니다.")
    logging.info("권장 전략:")
    logging.info("1. 서열 유사성 기반 분할: CD-HIT, MMseqs2 등 사용...")
    logging.info("2. 항원 기반 분할: 동일 항원 결합 항체는 동일 세트에...")
    logging.info("3. 클론형 인식 분할: V(D)J 유전자, CDR3 유사성 기반...")
    logging.info("\n구현 예시 방향 (2단계에서 수행):")
    logging.info("\n--- 예시 1: CD-HIT 사용 (CDR-H3 기준) ---")
    logging.info("\n--- 예시 2: 항원 기반 분할 (scikit-learn GroupShuffleSplit) ---")
    logging.info("="*50)
    logging.info(" 실제 데이터 분할은 모델링 단계(2단계)에서 수행하세요. ")
    logging.info("="*50)

In [ ]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------
# 10. 메인 파이프라인 실행 - v3 (AACDB 호출 수정)
# -----------------------------------------------------------
import asyncio
import time
import logging

async def run_pipeline():
    """전체 데이터 수집 및 전처리 파이프라인을 순차적으로 실행합니다."""
    pipeline_start_time = time.time()
    logging.info("#############################################")
    logging.info("####### 데이터 파이프라인 실행 시작 (AbNumber + Multi-Source) #######")
    logging.info("#############################################")

    # 0. 디렉토리 설정
    setup_directories()

    # 1. 데이터 소스별 처리 함수 호출
    absd_metadata = []
    sabdab_metadata = []
    aacdb_metadata = []
    oas_metadata = []

    logging.info("--- 데이터 소스 처리 시작 ---")
    # 비동기 함수들 실행 (SAbDab) - AACDB는 이제 동기 함수로 간주
    # SAbDab/PDB 처리
    await process_sabdab(sabdab_metadata)

    # 동기 함수들 순차 실행 (ABSD, OAS, AACDB)
    process_absd(absd_metadata)
    process_oas(oas_metadata)
    # --- 수정된 AACDB 호출 ---
    # process_aacdb 함수는 이제 내부적으로 상호작용 데이터를 파싱하므로 인자 불필요
    process_aacdb(aacdb_metadata)
    # --- 수정 끝 ---

    logging.info("--- 데이터 소스 처리 완료 ---")

    # 2. 데이터 통합 및 전처리
    final_processed_df = preprocess_and_integrate_data(
        absd_metadata,
        sabdab_metadata,
        aacdb_metadata,
        oas_metadata
    )

    # 3. 기본 특성 공학 (선택적)
    if final_processed_df is not None and not final_processed_df.empty:
        generate_basic_features(final_processed_df)
    else:
        logging.warning("전처리된 데이터가 없거나 비어있어 특성 공학을 건너<0xEB><0x9B><0x84>니다.")

    # 4. 데이터 분할 가이드 표시
    guide_data_splitting(final_processed_df)

    # 파이프라인 종료 로깅
    pipeline_end_time = time.time()
    total_time = pipeline_end_time - pipeline_start_time
    logging.info("#############################################")
    logging.info("####### 데이터 파이프라인 실행 완료 (AbNumber + Multi-Source) #######")
    logging.info(f" 총 소요 시간: {time.strftime('%H:%M:%S', time.gmtime(total_time))} ")
    if final_processed_df is not None and not final_processed_df.empty:
         logging.info(f" 최종 처리된 항체 쌍 데이터 수: {len(final_processed_df)} ")
         logging.info(f" 최종 메타데이터: {FINAL_METADATA_FILE} ")
         logging.info(f" 최종 서열 데이터: {os.path.join(FINAL_SEQS_DIR, 'final_antibody_seqs_abnumber.fasta')} ")
         logging.info(f" 최종 구조 데이터: {FINAL_PDB_DIR} ")
    else: logging.warning("최종 처리된 데이터프레임이 생성되지 않았거나 비어있습니다.")
    logging.info(f" 상세 로그 파일: {LOG_FILE} ")
    logging.info("#############################################")

# --- 비동기 파이프라인 실행 ---
# process_sabdab 만 await으로 남겨두고, 나머지는 동기적으로 실행
await run_pipeline()

# 만약 top-level await가 지원되지 않는 환경이라면 아래처럼 실행:
# asyncio.run(run_pipeline())

2025-04-27 12:36:56,588 - INFO     - #############################################
2025-04-27 12:36:56,591 - INFO     - ####### 데이터 파이프라인 실행 시작 (AbNumber + Multi-Source) #######
2025-04-27 12:36:56,592 - INFO     - #############################################
2025-04-27 12:36:56,594 - INFO     - 출력 디렉토리 설정 중...
2025-04-27 12:36:57,190 - INFO     - 출력 디렉토리 설정 완료.
2025-04-27 12:36:57,192 - INFO     - --- 데이터 소스 처리 시작 ---
2025-04-27 12:36:57,194 - INFO     - ==============================
2025-04-27 12:36:57,195 - INFO     -  SAbDab / PDB 데이터 처리 시작 
2025-04-27 12:36:57,197 - INFO     - ==============================
2025-04-27 12:36:57,199 - INFO     - 기존 SAbDab 요약 파일 사용: /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/sabdab/sabdab_summary.tsv
2025-04-27 12:36:57,200 - INFO     - SAbDab 요약 파일 로딩 중: /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/sabdab/sabdab_summary.tsv
2025-04-27 12:36:58,813 - INFO     - SAbDab 요약 파일 로드 완료. 18687개 항목.
2025-04-27 

SAbDab 항목 처리 중:   0%|          | 0/8322 [00:00<?, ?it/s]

2025-04-27 12:37:42,056 - DEBUG    - 9kat.pdb: L chain 'a' 찾을 수 없음.
2025-04-27 12:37:42,115 - DEBUG    - 9kat.pdb: L chain 'b' 찾을 수 없음.
2025-04-27 12:37:42,778 - DEBUG    - 8vfx.pdb: L chain 'm' 찾을 수 없음.
2025-04-27 12:37:43,090 - DEBUG    - 8vfx.pdb: L chain 'n' 찾을 수 없음.
2025-04-27 12:37:59,762 - DEBUG    - 8ywe.pdb: L chain 'b' 찾을 수 없음.
2025-04-27 12:38:00,049 - DEBUG    - 8ywe.pdb: L chain 'c' 찾을 수 없음.
2025-04-27 12:38:01,048 - DEBUG    - 9as8.pdb: L chain 'e' 찾을 수 없음.
2025-04-27 12:38:02,827 - DEBUG    - 9izc.pdb: L chain 's' 찾을 수 없음.
2025-04-27 12:38:03,381 - DEBUG    - 9k6l.pdb: L chain 's' 찾을 수 없음.
2025-04-27 12:38:34,914 - DEBUG    - 8jsr.pdb: L chain 'c' 찾을 수 없음.
2025-04-27 12:38:37,833 - DEBUG    - 8zf9.pdb: L chain 's' 찾을 수 없음.
2025-04-27 12:38:44,481 - DEBUG    - 8uuj.pdb: L chain 'd' 찾을 수 없음.
2025-04-27 12:38:46,858 - DEBUG    - 8x9t.pdb: L chain 's' 찾을 수 없음.
2025-04-27 12:38:47,549 - DEBUG    - 8y01.pdb: L chain 's' 찾을 수 없음.
2025-04-27 12:38:59,558 - DEBUG    - 8zd1.pdb: L

ABSD FASTA 파싱 중: 0it [00:00, ?it/s]

2025-04-27 13:43:25,501 - INFO     - ABSD 파싱 완료. 10000개 서열 항목 처리.
2025-04-27 13:43:25,512 - INFO     - ABSD 데이터 처리 완료.
2025-04-27 13:43:25,518 - INFO     - ==============================
2025-04-27 13:43:25,523 - WARNING  -  OAS 데이터 처리 시작 (v3.1: 로컬 CSV 파일 파싱 및 페어링) 
2025-04-27 13:43:25,526 - INFO     - ==============================
2025-04-27 13:43:25,528 - WARNING  - OAS 데이터 디렉토리(/content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/0_raw_data/oas)에서 .csv.gz 파일을 검색합니다.
2025-04-27 13:43:25,535 - WARNING  - bulk_download.sh 스크립트가 이 디렉토리에 파일을 다운로드했어야 합니다.
2025-04-27 13:43:25,593 - INFO     - 310개의 OAS .csv.gz 파일 처리 시작...


...

Streaming output truncated to the last 5000 lines.
2025-04-27 14:48:56,941 - DEBUG    - Extracted 107 residues via chain.positions
2025-04-27 14:48:56,942 - DEBUG    - AbNumber result: Status='Success', SeqLen=107, NumRes=107, CDRs=True
2025-04-27 14:48:56,981 - DEBUG    - Chain type: H, Seq: EVQLVESGGGLVQPGGSLRLSCAASGFTFS...
2025-04-27 14:48:56,985 - DEBUG    - Extracted 120 residues via chain.positions
2025-04-27 14:48:56,986 - DEBUG    - AbNumber result: Status='Success', SeqLen=120, NumRes=120, CDRs=True
2025-04-27 14:48:57,061 - DEBUG    - Chain type: K, Seq: DIQLTQSPSFLSASVGDRVTITCRASQGIS...
2025-04-27 14:48:57,063 - DEBUG    - Extracted 107 residues via chain.positions
2025-04-27 14:48:57,064 - DEBUG    - AbNumber result: Status='Success', SeqLen=107, NumRes=107, CDRs=True
2025-04-27 14:48:57,108 - DEBUG    - Chain type: H, Seq: QVQLQESGPGLVKPSQTLSLTCTVSGDSIT...
2025-04-27 14:48:57,111 - DEBUG    - Extracted 122 residues via chain.positions
2025-04-27 14:48:57,114 - DEBUG    - A

FASTA 저장:   0%|          | 0/10594 [00:00<?, ?it/s]

2025-04-27 14:50:47,019 - INFO     - 10594개 항체 쌍 (VH/VL) 서열 저장 완료: /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/1_preprocessed/final_dataset/sequences/final_antibody_seqs_abnumber.fasta
2025-04-27 14:50:47,030 - INFO     - 관련 PDB 구조 파일 정리 중...


PDB 파일 정리:   0%|          | 0/622 [00:00<?, ?it/s]

2025-04-27 14:58:52,745 - INFO     - 622개의 관련 PDB 파일 저장/복사 완료: /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/1_preprocessed/final_dataset/structures
2025-04-27 14:58:52,748 - INFO     - 데이터 통합 및 전처리 완료.
2025-04-27 14:58:52,777 - INFO     - ==============================
2025-04-27 14:58:52,780 - INFO     -  기본 특성 공학 시작 (One-Hot Encoding 예시) 
2025-04-27 14:58:52,782 - INFO     - ==============================
2025-04-27 14:58:52,784 - INFO     - One-Hot Encoding (간단 예시 - 실제 사용 시 보완 필요)
2025-04-27 14:58:52,785 - WARNING  - One-Hot Encoding은 기본적이며, 서열/구조적 문맥 정보를 반영하지 못합니다.
2025-04-27 14:58:52,786 - INFO     - ==============================
2025-04-27 14:58:52,789 - INFO     -  향후 특성 공학 방향 (2단계 수행 권장) 
2025-04-27 14:58:52,790 - INFO     - ==============================
2025-04-27 14:58:52,792 - INFO     - 1. 단백질 언어 모델 (pLM) 임베딩: Hugging Face transformers (ESM, ProtBERT, AntiBERTy 등)...
2025-04-27 14:58:52,793 - INFO     - 2. 구조 기반 특성 (GNN 입력용): BioPython, PyTorch Geometric (PyG).

In [ ]:
# -*- coding: utf-8 -*-
# -----------------------------------------------------------
# 11. 최종 결과 확인 (간단 검증 - AbNumber + Multi-Source 버전)
# -----------------------------------------------------------
import pandas as pd
import os
import logging
from IPython.display import display # Colab display

logging.info("="*30)
logging.info(" 최종 결과 확인 (AbNumber + Multi-Source 버전) ")
logging.info("="*30)

# 최종 파일 경로 (셀 1에서 정의된 변수 사용)
final_meta_path = FINAL_METADATA_FILE
final_csv_path = os.path.splitext(final_meta_path)[0] + ".csv"
final_seq_path = os.path.join(FINAL_SEQS_DIR, "final_antibody_seqs_abnumber.fasta")
final_pdb_dir = FINAL_PDB_DIR

# 메타데이터 파일 로드 및 분석
try:
    final_df = None
    if os.path.exists(final_meta_path):
        final_df = pd.read_parquet(final_meta_path)
        logging.info(f"최종 메타데이터 로드 성공: {final_meta_path} ({len(final_df)} 항목)")
    elif os.path.exists(final_csv_path):
         final_df = pd.read_csv(final_csv_path)
         logging.info(f"CSV 메타데이터 로드 성공: {final_csv_path} ({len(final_df)} 항목)")
    else:
         logging.error(f"최종 메타데이터 파일({final_meta_path} 또는 {final_csv_path}) 없음.")

    if final_df is not None and not final_df.empty:
        print("\n--- 최종 메타데이터 샘플 (상위 5개) ---"); display(final_df.head())
        print(f"\n--- 최종 데이터 컬럼 ({len(final_df.columns)}개) ---"); print(list(final_df.columns))
        print("\n--- 데이터 소스 분포 ---"); print(final_df['source_database'].value_counts().to_markdown())

        # 종 분포 확인 (여러 컬럼 확인)
        species_cols = ['species', 'heavy_species', 'light_species', 'organism_antibody']
        for col in species_cols:
             if col in final_df.columns:
                 print(f"\n--- 종(Species) 분포 ('{col}' 컬럼 기준, 상위 10개) ---")
                 print(final_df[col].fillna('Unknown').value_counts().head(10).to_markdown()); break # 하나만 출력

        # CDR3 길이 분포 (VH)
        if 'vh_cdr3' in final_df.columns:
            print("\n--- CDR3 길이 분포 (VH) ---")
            vh_cdr3_lengths = final_df['vh_cdr3'].dropna().astype(str).str.len()
            if not vh_cdr3_lengths.empty: print(vh_cdr3_lengths.describe().to_markdown())
            else: print("VH CDR3 데이터 없음.")
        else: print("VH CDR3 컬럼 없음.")

        # AbNumber 상태 요약
        if 'abnumber_status_vh' in final_df.columns: print("\n--- AbNumber VH 처리 상태 ---"); print(final_df['abnumber_status_vh'].value_counts().to_markdown())
        if 'abnumber_status_vl' in final_df.columns: print("\n--- AbNumber VL 처리 상태 ---"); print(final_df['abnumber_status_vl'].value_counts().to_markdown())

except Exception as e: logging.error(f"최종 메타데이터 파일 로드/분석 중 오류: {e}"); import traceback; traceback.print_exc()

# 서열 파일 확인
if os.path.exists(final_seq_path):
     logging.info(f"최종 서열 파일 확인: {final_seq_path}")
     try: logging.info(f"  - 파일 크기: {os.path.getsize(final_seq_path) / (1024*1024):.2f} MB")
     except Exception as e: logging.error(f"서열 파일 정보 읽기 오류: {e}")
else: logging.warning(f"최종 서열 파일({final_seq_path}) 없음.")

# 구조 파일 디렉토리 확인
if os.path.exists(final_pdb_dir):
    try:
        pdb_files = [f for f in os.listdir(final_pdb_dir) if f.endswith('.pdb')]
        logging.info(f"최종 구조 디렉토리 확인: {final_pdb_dir} ({len(pdb_files)}개 PDB 파일)")
        if pdb_files: print("\n--- 구조 파일 예시 (최대 10개) ---"); print(pdb_files[:10])
    except Exception as e: logging.error(f"구조 디렉토리 확인 중 오류: {e}")
else: logging.warning(f"최종 구조 디렉토리({final_pdb_dir}) 없음.")

logging.info("="*30); logging.info(" 결과 확인 완료 "); logging.info("="*30)

2025-04-27 14:58:52,896 - INFO     - ==============================
2025-04-27 14:58:52,898 - INFO     -  최종 결과 확인 (AbNumber + Multi-Source 버전) 
2025-04-27 14:58:52,899 - INFO     - ==============================
2025-04-27 14:58:53,164 - INFO     - 최종 메타데이터 로드 성공: /content/drive/MyDrive/Antibody_AI_Data_Stage1_AbNumber/1_preprocessed/final_dataset/antibody_metadata_abnumber.parquet (10594 항목)

--- 최종 메타데이터 샘플 (상위 5개) ---


,entry_id,source_database,source_id,pdb_id,h_chain_id,l_chain_id,vh_sequence,vl_sequence,vh_cdr1,vh_cdr2,...,engineered,header,raw_pdb_path,abnumber_status_vh,abnumber_status_vl,v_call_heavy,j_call_heavy,v_call_light,j_call_light,isotype_heavy
0,SAbDab_9b6t_H_L,SAbDab,9B6T,9B6T,H,L,QVQLQESGPGLVKPSETLSLTCTVSGDSIRSYYWSWIRQPPGKGLE...,ALTQPPSASGTPGQRVTISCSGSSSNIGSNTVNWYQQLPGTAPKLL...,GDSIRSYY,IYYSGST,...,True,None,/content/drive/MyDrive/Antibody_AI_Data_Stage1...,Success,Success,None,None,None,None,None
1,SAbDab_9b7p_H_L,SAbDab,9B7P,9B7P,H,L,VQLLESGGALVQPGGSLRLSCAASGFSFSNYAMSWVRQAPGKGLEW...,EIVLTQSPGTLSLSPGERATLSCRASQSVRSSYLAWYQQKPGQAPR...,GFSFSNYA,ISASGGTT,...,True,None,/content/drive/MyDrive/Antibody_AI_Data_Stage1...,Success,Success,None,None,None,None,None
2,SAbDab_9dhy_A_B,SAbDab,9DHY,9DHY,A,B,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQTPGKGLE...,IQMTQSPSSLSTSVGDRVTITCRASQGIRNDLGWYQLKPGKAPKLL...,GFTFDDYA,ISWNSGSI,...,True,None,/content/drive/MyDrive/Antibody_AI_Data_Stage1...,Success,Success,None,None,None,None,None
3,SAbDab_8we4_H_L,SAbDab,8WE4,8WE4,H,L,VQLVESGGGLVQPGGSLRLSCAASGFTFSSYDMHWVRQTTGKGLEW...,DIEMTQSPSSLSAAVGDRVTITCRASQSIGSYLNWYQQKPGKAPKL...,GFTFSSYD,IGTAGDT,...,True,None,/content/drive/MyDrive/Antibody_AI_Data_Stage1...,Success,Success,None,None,None,None,None
4,SAbDab_9fjk_H_L,SAbDab,9FJK,9FJK,H,L,QVQLQESGPGLVKPSETLSLTCTVSGGSISSRSYYWGWIRQPPGKG...,QSALTQPPSVSGAPGQRVAISCTGSSANIGTADDVHWYQQLPRTAP...,GGSISSRSYY,IYYSGST,...,True,None,/content/drive/MyDrive/Antibody_AI_Data_Stage1...,Success,Success,None,None,None,None,None



--- 최종 데이터 컬럼 (41개) ---
['entry_id', 'source_database', 'source_id', 'pdb_id', 'h_chain_id', 'l_chain_id', 'vh_sequence', 'vl_sequence', 'vh_cdr1', 'vh_cdr2', 'vh_cdr3', 'vl_cdr1', 'vl_cdr2', 'vl_cdr3', 'sequence_source', 'chain_type', 'species', 'heavy_species', 'light_species', 'antigen_species', 'resolution', 'exp_method', 'affinity', 'affinity_method', 'temperature', 'antigen_name', 'pmid', 'deposition_date', 'structure_header', 'compound_info', 'scfv', 'engineered', 'header', 'raw_pdb_path', 'abnumber_status_vh', 'abnumber_status_vl', 'v_call_heavy', 'j_call_heavy', 'v_call_light', 'j_call_light', 'isotype_heavy']

--- 데이터 소스 분포 ---
| source_database   |   count |
|:------------------|--------:|
| OAS               |    9972 |
| SAbDab            |     622 |

--- 종(Species) 분포 ('species' 컬럼 기준, 상위 10개) ---
| species   |   count |
|:----------|--------:|
| Unknown   |   10594 |

--- CDR3 길이 분포 (VH) ---
|       |     vh_cdr3 |
|:------|------------:|
| count | 10594       |
| mean 